In [1]:
import os
import os.path as osp
import mmcv
import torch
from mmcv.cnn.utils import revert_sync_batchnorm
from mmcv.runner import (get_dist_info, load_checkpoint, wrap_fp16_model)

from mmseg.apis import single_gpu_test
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.utils import build_dp, get_device, setup_multi_processes

import pvt, pvtv2

C:\Users\nexty\anaconda3\envs\PVT\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['LOCAL_RANK'] = '0'


In [3]:
# указываем пути конфига, весов модели, метрики для теста и надо ли отрисовывать разметку и с какой прозрачностью
cfg_path = '../my_configs/fpn_pvtv2_b5_casia512_320k.py'
checkpoint_path = '../work_dirs/casia512_320k/fpn_pvtv2_b5/iter_165000.pth'
cfg = mmcv.Config.fromfile(cfg_path)
opacity = 1
show = True
show_dir = '../work_dirs/casia512_320k/fpn_pvtv2_b5/preds'
eval_metrics = ['mIoU', 'mFscore', 'mDice']
cfg.work_dir = '../work_dirs/casia512_320k/fpn_pvtv2_b5'
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
json_file = osp.join(cfg.work_dir, f'eval_single_scale.json')

In [4]:
# Предварительные настройки
setup_multi_processes(cfg)
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True
cfg.model.pretrained = None
cfg.data.test.test_mode = True
distributed = False
rank, _ = get_dist_info()
cfg.gpu_ids = range(1)
cfg.device = get_device()


2023-07-29 00:37:00,415 - mmseg - INFO - OpenCV num_threads is `16


In [5]:
# Строим датасет
dataset = build_dataset(cfg.data.test)
loader_cfg = dict(
        num_gpus=len(cfg.gpu_ids),
        dist=distributed,
        shuffle=False)

loader_cfg.update({
        k: v
        for k, v in cfg.data.items() if k not in [
            'train', 'val', 'test', 'train_dataloader', 'val_dataloader',
            'test_dataloader'
        ]
})
test_loader_cfg = {
    **loader_cfg,
    'samples_per_gpu': 1,
    'shuffle': False,
    **cfg.data.get('test_dataloader', {})
}
data_loader = build_dataloader(dataset, **test_loader_cfg)

2023-07-29 00:37:00,437 - mmseg - INFO - Loaded 511 images


In [6]:
# Инициируем модель
cfg.model.train_cfg = None
model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
fp16_cfg = cfg.get('fp16', None)
if fp16_cfg is not None:
    wrap_fp16_model(model)
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES
model.PALETTE = dataset.PALETTE

torch.cuda.empty_cache()
model = revert_sync_batchnorm(model)
model = build_dp(model, cfg.device, device_ids=cfg.gpu_ids)


c:\users\nexty\desktop\pvtv2-seg\mmsegmentation\mmseg\models\decode_heads\decode_head.py:120: UserWarning: threshold is not defined for binary, and defaultsto 0.3
  warnings.warn('threshold is not defined for binary, and defaults'
c:\users\nexty\desktop\pvtv2-seg\mmsegmentation\mmseg\models\losses\cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: ../work_dirs/casia512_320k/fpn_pvtv2_b5/iter_165000.pth


In [ ]:
# Процесс тестирования
eval_kwargs = {}
results = single_gpu_test(
            model,
            data_loader,
            show,
            show_dir,
            False,
            opacity,
            pre_eval=eval_metrics,
            format_only=False,
            format_args=eval_kwargs)

eval_kwargs.update(metric=eval_metrics)
metric = dataset.evaluate(results, **eval_kwargs)
metric_dict = dict(config=cfg_path, metric=metric)
mmcv.dump(metric_dict, json_file, indent=4)


[                                                  ] 1/511, 0.2 task/s, elapsed: 6s, ETA:  2845s

C:\Users\nexty\anaconda3\envs\PVT\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 504/511, 4.5 task/s, elapsed: 111s, ETA:     2s